In [1]:
import pickle
import numpy as np
from summary_stats import compute_rmse
import pyabc

In [2]:
obs_pars = {
    'gradient_strength': 100.,  # strength of the gradient of chemotaxis
    'move.strength': 10.,  # strength of directed motion
    'move.duration.mean': 0.1,  # mean of exponential distribution (seconds)
    'cell_nodes_real': 50.,  # area of the cell (\mu m^2), macrophages have a volume of 4990\mu m^3 -> radius of 17 if they would are sphere
}

obs_pars_log = {key: np.log10(val) for key, val in obs_pars.items()}
limits = {'gradient_strength': (1, 10000), #(10 ** 4, 10 ** 8),
          'move.strength': (1, 100),
          'move.duration.mean': (1e-4, 30), #(math.log10((10 ** -2) * 30), math.log10((10 ** 4))), # smallest time step in simulation 5
          'cell_nodes_real': (1, 300)}
limits_log = {key: (np.log10(val[0]), np.log10(val[1])) for key, val in limits.items()}


prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc=lb, scale=ub-lb)
                              for key, (lb, ub) in limits_log.items()})

In [3]:
# load test data
posterior_samples_list = []
test_params = []

for test_id in [0,1,2]:
    np.random.seed(42+test_id)
    test_params_i = np.array(list(prior.rvs().values()))
    test_sim_i = np.load(f'test_sim_{test_id}.npy')

    with open(f'posterior_samples_{test_id}.pickle', 'rb') as f:
        posterior_samples_i = pickle.load(f)

    posterior_samples_list.append(posterior_samples_i)
    test_params.append(test_params_i)

test_params = np.stack(test_params, axis=0)
posterior_samples = {}
for method in posterior_samples_list[0].keys():
    posterior_samples[method] = np.stack([posterior_samples_list[i][method] for i in range(3)], axis=0)

In [6]:
for method in posterior_samples.keys():
    if method == 'npe':
        continue  # npe is computed on more data
    rmse_params = compute_rmse(test_params, posterior_samples[method], normalize=True)
    print(method, rmse_params.mean())

abc 0.22968104065206413
abc_mean 0.49376087474125724
abc_npe 0.23162118736364878
